In [2]:
3 + 2 #Shift + Enter

5

In [3]:
import xlwings

In [4]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [5]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [6]:
xlwings.Range("A1").expand()

<Range [통합 문서1]Sheet1!$A$1:$B$10>

In [7]:
xlwings.Range("A1").expand().value

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [11]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [12]:
colum_names = total_rows[0]
colum_names

['message', 'browser']

In [13]:
total_rows[1:]

[['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [14]:
for row in total_rows[1:]:
    row_dict(zip(colum_names, row))
    print(row_dict)

NameError: name 'row_dict' is not defined

In [15]:
for row in total_rows[1:]:
    row_dict(zip(colum_names, row))
    print(row_dict)

NameError: name 'row_dict' is not defined

In [16]:
# list comprehension
rows_data = [
    dict(zip(colum_names, row))
    for row in total_rows[1:]
]
rows_data

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

In [18]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 데이터베이스 접속

In [21]:
DATABASE = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "1234",
    "db": "myproj11_db",
    "charset":"utf8"
}

In [24]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log")
print(cursor.fetchall())
cursor.close()

()


# with절

In [25]:
import pymysql
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [26]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    print(cursor.fetchall())

()


In [33]:
colum_data = {'message': '첫 로그','browser':'chrome'}

with db_cursor() as cursor:
    for row_dict in get_rows_data():
        sql = "INSERT INTO LOG(message, browser) VALUES(%(message)s, %(browser)s)"
        cursor.execute(sql, ["첫 로그", "chrome"])

TypeError: format requires a mapping

In [66]:
with db_cursor() as cursor:
    sql = 'select * from log where id = 1 or 1'
    cursor.execute(sql)
    for row in cursor:
        print(row)
row

(4, '첫 로그', 'chrome')
(5, '두번째 로그', 'chrome')
(6, '세번째 로그', 'chrome')


(6, '세번째 로그', 'chrome')

In [65]:
with db_cursor() as cursor:
#     log_id = '1 or 1'
#     sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
    # sql = "SELECT * FROM LOG WHERE id = 1 or 1"
    
    sql = "SELECT * FROM log WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

In [27]:
get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 데이터 매핑의 예

In [56]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#     for row in cursor:
#         row[0] # id
#         row[1] # message
#         row[2] # chrome
#         print(row)
rows_data

((4, '첫 로그', 'chrome'), (5, '두번째 로그', 'chrome'), (6, '세번째 로그', 'chrome'))

## dict 으로의 변환

## namedtuple

In [57]:
from collections import namedtuple

Log = namedtuple('Log','id message browser')

In [60]:
log = (1, 'message','chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [59]:
log = Log(1, 'message','chrome')

Log(id=1, message='message', browser='chrome')

In [62]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()]
rows_data

[Log(id=4, message='첫 로그', browser='chrome'),
 Log(id=5, message='두번째 로그', browser='chrome'),
 Log(id=6, message='세번째 로그', browser='chrome')]